In [10]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Activation
from keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

In [11]:
with open('content\sherlock-holmes.txt', 'r') as file:
    text = file.read().lower()
print('text length', len(text))
chars = sorted(list(set(text)))
print('total chars: ', len(chars))

text length 609265
total chars:  58


In [12]:
bad_chars = [';','½','£', 'à', 'â', 'è', 'é']
for i in range(len(bad_chars)):
  text = text.replace(bad_chars[i],"")
chars = sorted(list(set(text)))
textlen = len(text)
charlen = len(chars)
print("length is " + str(textlen))
print("length is " + str(charlen))
print(chars)

length is 609013
length is 51
['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [13]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
print(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [14]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [15]:
#Model building
model = Sequential()
model.add(LSTM(256, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

In [16]:
model.compile(loss='categorical_crossentropy',optimizer='adam')

In [17]:
import random
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [18]:
# Setting Checkpoints
filepath = "weights20.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss',
                             verbose=1, save_best_only=True,
                             mode='min')

In [19]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=1, min_lr=0.001)

callbacks = [checkpoint, reduce_lr]

In [20]:
model.fit(x, y, batch_size=128, epochs=20, callbacks=callbacks)

Epoch 1/4
1591/1591 [==============================] - ETA: 0s - loss: 2.2259
Epoch 1: loss improved from inf to 2.22594, saving model to weights3.hdf5
1591/1591 [==============================] - 196s 122ms/step - loss: 2.2259 - lr: 0.0010
Epoch 2/4
1590/1591 [============================>.] - ETA: 0s - loss: 1.8070
Epoch 2: loss improved from 2.22594 to 1.80697, saving model to weights3.hdf5
1591/1591 [==============================] - 205s 129ms/step - loss: 1.8070 - lr: 0.0010
Epoch 3/4
1591/1591 [==============================] - ETA: 0s - loss: 1.6499
Epoch 3: loss improved from 1.80697 to 1.64987, saving model to weights3.hdf5
1591/1591 [==============================] - 229s 144ms/step - loss: 1.6499 - lr: 0.0010
Epoch 4/4
1590/1591 [============================>.] - ETA: 0s - loss: 1.5473
Epoch 4: loss improved from 1.64987 to 1.54733, saving model to weights3.hdf5
1591/1591 [==============================] - 223s 140ms/step - loss: 1.5473 - lr: 0.0010


In [ ]:
def generate_text(length, diversity):
    # Function to get random starting text and generate text for it
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    for i in range(length):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
    return generated

In [1]:
print(generate_text(500, 0.3))